In [64]:
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
import nltk
import math
%matplotlib inline

In [65]:
def sigmoid(x):
    return 1 / (1 + math.e ** -x)

In [66]:
os.chdir(r'D:\Hackathon\How-To-Choose-The-Perfect-Beer-Data-Set')

In [67]:
train_data=pd.read_csv('train_cleaned.csv')
test_data=pd.read_csv('test_cleaned.csv')
train_data.shape, test_data.shape

((185643, 18), (20628, 18))

In [68]:
# Load the submission file and convert the scores to array
y_act_final=pd.read_excel('Beer_Sample_Submission.xlsx',sheet_name='Sheet1')
y_act_score=y_act_final.Score
y_act_score=np.array(y_act_score)

In [69]:
train_data['glassware_len']=[len(i) for i in train_data['Glassware Used']]
test_data['glassware_len']=[len(i) for i in test_data['Glassware Used']]

In [70]:
# Need to remove outliers
train_data['Ratings'].describe()

count    185643.000000
mean         32.955210
std         254.863803
min           0.000000
25%           1.000000
50%           2.000000
75%           7.000000
max       16954.000000
Name: Ratings, dtype: float64

In [71]:
train_data['ratings_cat']=pd.qcut(train_data['Ratings'], q=[0,0.25,0.5,0.75, 1], labels=['A', 'B','C', 'D'])
test_data['ratings_cat']=pd.qcut(test_data['Ratings'], q=[0,0.25,0.5,0.75, 1], labels=['A', 'B','C', 'D'])

In [72]:
# This looks reasonable
train_data['ABV'].describe()

count    185643.000000
mean          6.354961
std           1.827834
min           0.010000
25%           5.100000
50%           6.100000
75%           7.000000
max          80.000000
Name: ABV, dtype: float64

In [73]:
train_data.head()

,Unnamed: 0,ABV,Brewing Company,Food Paring,Glassware Used,Beer Name,Ratings,Style Name,Cellar Temperature,Serving Temperature,Score,style_len,min_cellar_temp,max_cellar_temp,min_serving_temp,max_serving_temp,item_score,glass_score,glassware_len,ratings_cat
0,0,6.500000,8929,"(Curried,Thai)Cheese(pepperyMontereyPepperJack...","PintGlass(orBecker,Nonic,Tumbler),Mug(orSeidel...",15121,22,AmericanIPA,40-45,45-50,3.28,11,40,45,45,50,3.0,2.0,53,D
1,1,5.500000,13187,"(PanAsian)Cheese(earthyCamembert,Fontina,nutty...","PintGlass(orBecker,Nonic,Tumbler),Mug(orSeidel...",59817,1,AmericanPaleAle(APA),35-40,40-45,3.52,20,35,40,40,45,4.0,2.0,53,A
2,2,8.100000,6834,"Meat(Pork,Poultry)","PintGlass(orBecker,Nonic,Tumbler),Mug(orSeidel...",32669,3,IrishRedAle,35-40,40-45,4.01,11,35,40,40,45,1.0,2.0,53,C
3,3,6.354961,11688,"(Indian,LatinAmerican,PanAsian)General(Aperitif)","PintGlass(orBecker,Nonic,Tumbler),PilsenerGlas...",130798,0,AmericanMaltLiquor,35-40,35-40,0.00,18,35,40,35,40,2.0,3.0,76,A
4,4,6.000000,10417,"Meat(Poultry,Fish,Shellfish)",PilsenerGlass(orPokal),124087,1,EuroPaleLager,35-40,40-45,2.73,13,35,40,40,45,1.0,1.0,22,A


In [74]:
test_data.head()

,Unnamed: 0,ABV,Brewing Company,Food Paring,Glassware Used,Beer Name,Ratings,Style Name,Cellar Temperature,Serving Temperature,Score,style_len,min_cellar_temp,max_cellar_temp,min_serving_temp,max_serving_temp,item_score,glass_score,glassware_len,ratings_cat
0,0,6.2,8803,"(Curried,Thai)Cheese(pepperyMontereyPepperJack...","PintGlass(orBecker,Nonic,Tumbler),Mug(orSeidel...",34558,3,AmericanIPA,40-45,45-50,NaN,11,40,45,45,50,3.0,2.0,53,C
1,1,5.3,8558,"(Barbecue)Cheese(butteryBrie,Gouda,Havarti,Swi...","PintGlass(orBecker,Nonic,Tumbler),Mug(orSeidel...",86826,20,AmericanPorter,40-45,45-50,NaN,14,40,45,45,50,4.0,2.0,53,D
2,2,7.0,7519,"Cheese(earthyCamembert,Fontina)General(Aperitif)","Snifter,Tulip,OversizedWineGlass",17051,0,BelgianDarkAle,45-50,45-50,NaN,14,45,50,45,50,2.0,3.0,32,A
3,3,5.2,9852,"(LatinAmerican,German)Meat(Pork,Poultry)","Flute,PilsenerGlass(orPokal),Mug(orSeidel,Stein)",49156,2,ViennaLager,35-40,40-45,NaN,11,35,40,40,45,2.0,3.0,48,B
4,4,8.1,8991,"(Barbecue)Cheese(butteryBrie,Gouda,Havarti,Swi...","PintGlass(orBecker,Nonic,Tumbler),Mug(orSeidel...",162723,5,AmericanPorter,40-45,45-50,NaN,14,40,45,45,50,4.0,2.0,53,C


In [75]:
train_data_1=pd.get_dummies(train_data['ratings_cat'])
test_data_1=pd.get_dummies(test_data['ratings_cat'])

In [76]:
train_data=train_data.join(train_data_1)
test_data=test_data.join(test_data_1)

In [77]:
train_data_final=train_data.drop(labels=['Unnamed: 0','Beer Name','Brewing Company','max_cellar_temp',\
                                        'max_serving_temp','glassware_len','Food Paring','Glassware Used','Style Name',\
                                        'Cellar Temperature','Serving Temperature','ratings_cat'], axis=1)
test_data_final=test_data.drop(labels=['Unnamed: 0','Beer Name','Brewing Company','max_cellar_temp',\
                                        'max_serving_temp','glassware_len','Food Paring','Glassware Used','Style Name',\
                                        'Cellar Temperature','Serving Temperature','ratings_cat'], axis=1)

In [78]:
train_data_final.head()

,ABV,Ratings,Score,style_len,min_cellar_temp,min_serving_temp,item_score,glass_score,A,B,C,D
0,6.500000,22,3.28,11,40,45,3.0,2.0,0,0,0,1
1,5.500000,1,3.52,20,35,40,4.0,2.0,1,0,0,0
2,8.100000,3,4.01,11,35,40,1.0,2.0,0,0,1,0
3,6.354961,0,0.00,18,35,35,2.0,3.0,1,0,0,0
4,6.000000,1,2.73,13,35,40,1.0,1.0,1,0,0,0


In [79]:
train_data_final.corr()

,ABV,Ratings,Score,style_len,min_cellar_temp,min_serving_temp,item_score,glass_score,A,B,C,D
ABV,1.000000,0.053559,0.130498,0.272474,0.288131,0.461440,0.029844,0.302961,-0.071572,-0.001701,0.009918,0.073951
Ratings,0.053559,1.000000,0.065293,0.023099,0.022313,0.028100,-0.007520,0.025692,-0.107457,-0.046509,-0.057415,0.213216
Score,0.130498,0.065293,1.000000,0.052274,0.074949,0.124193,0.036020,0.077039,-0.467859,0.146154,0.198184,0.235860
style_len,0.272474,0.023099,0.052274,1.000000,0.139246,0.169907,0.057706,0.338820,-0.040906,-0.006085,0.012945,0.039376
min_cellar_temp,0.288131,0.022313,0.074949,0.139246,1.000000,0.266265,0.224390,0.196104,-0.042746,-0.003859,0.012279,0.040386
min_serving_temp,0.461440,0.028100,0.124193,0.169907,0.266265,1.000000,0.101435,0.290187,-0.046020,0.004789,0.010756,0.038863
item_score,0.029844,-0.007520,0.036020,0.057706,0.224390,0.101435,1.000000,-0.007843,0.002632,0.001384,0.006163,-0.009892
glass_score,0.302961,0.025692,0.077039,0.338820,0.196104,0.290187,-0.007843,1.000000,-0.039314,-0.004783,0.008000,0.041201
A,-0.071572,-0.107457,-0.467859,-0.040906,-0.042746,-0.046020,0.002632,-0.039314,1.000000,-0.324767,-0.434264,-0.484482
B,-0.001701,-0.046509,0.146154,-0.006085,-0.003859,0.004789,0.001384,-0.004783,-0.324767,1.000000,-0.196068,-0.218742


In [89]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

# Model Packages
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

def model_eval(train_data, regressor, final_test_set, actual_score):
    y=train_data['Score']
    X=train_data.drop(labels=['Score'], axis=1)
    
    # Split the train data to train-test split
    X_train, X_test, y_train, y_test=train_test_split(X, y, train_size=0.8, random_state=45)
    
    # Training model object
    print('Training the model with 80-20 Split.....')
    regr=regressor
    model_obj=regr.fit(X_train, y_train)
    y_pred=model_obj.predict(X_test)
    y_act=y_test
    mean_abs_err=mean_absolute_error(y_act, y_pred)
    root_mean_sqr_err=sqrt(mean_squared_error(y_act, y_pred))
    print('RMSE_Train_model: ', root_mean_sqr_err)
    sigmoid_value=sigmoid(root_mean_sqr_err)
    print('Score_trained_model: ',(1-sigmoid_value))
    
    # Final test set
    pred_score=model_obj.predict(final_test_set.drop(labels='Score', axis=1))
    root_mean_sqr_err_final=sqrt(mean_squared_error(actual_score, pred_score))
    print('RMSE_Test_set: ', root_mean_sqr_err_final)
    sigmoid_value_test=sigmoid(root_mean_sqr_err_final)
    print('Score_on_test_set: ',(1-sigmoid_value_test))

In [90]:
model_eval(train_data=train_data_final, regressor=LinearRegression(), final_test_set=test_data_final,\
           actual_score=y_act_score)

Training the model with 80-20 Split.....
RMSE_Train_model:  1.1862212444393614
Score_trained_model:  0.23393544427933521
RMSE_Test_set:  1.7096756053869087
Score_on_test_set:  0.1532057959701557


C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [91]:
model_eval(train_data=train_data_final, regressor=RandomForestRegressor(), final_test_set=test_data_final,\
           actual_score=y_act_score)

C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Training the model with 80-20 Split.....
RMSE_Train_model:  0.379314885625666
Score_trained_model:  0.406292148943844
RMSE_Test_set:  2.061413266802903
Score_on_test_set:  0.11290420427245373


In [85]:
model_eval(train_data=train_data_final, regressor=GradientBoostingRegressor(), final_test_set=test_data_final,\
           actual_score=y_act_score)

C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Training the model with 80-20 Split.....
RMSE_Train_model:  0.3670148730697786
Score_trained_model:  0.4092625299230216
RMSE_Test_set:  2.0503740662658374
Score_on_test_set:  0.11401458942153131


In [92]:
model_eval(train_data=train_data_final, regressor=XGBRegressor(), final_test_set=test_data_final,\
           actual_score=y_act_score)

C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Training the model with 80-20 Split.....
RMSE_Train_model:  0.36714119973700055
Score_trained_model:  0.40923198869023103
RMSE_Test_set:  2.050391863134502
Score_on_test_set:  0.11401279167851519


In [61]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

y=train_data_final['Score']
X=train_data_final.drop(labels=['Score'], axis=1)

# Split the train data to train-test split
X_train, X_test, y_train, y_test=train_test_split(X, y, train_size=0.8, random_state=45)

# Training model object
regressor=LinearRegression()
model_obj=regressor.fit(X_train, y_train)
y_pred=model_obj.predict(X_test)
y_act=y_test
mean_abs_err=mean_absolute_error(y_act, y_pred)
root_mean_sqr_err=sqrt(mean_squared_error(y_act, y_pred))

print('Mean absolute error: ', mean_abs_err)
print('Root Mean squared error: ', root_mean_sqr_err)

Mean absolute error:  0.851313921177971
Root Mean squared error:  1.1862212444393614


C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

y=train_data_final['Score']
X=train_data_final.drop(labels=['Score'], axis=1)

# Split the train data to train-test split
X_train, X_test, y_train, y_test=train_test_split(X, y, train_size=0.8, random_state=45)

# Training model object
regressor=RandomForestRegressor()
model_obj=regressor.fit(X_train, y_train)
y_pred=model_obj.predict(X_test)
y_act=y_test
mean_abs_err=mean_absolute_error(y_act, y_pred)
root_mean_sqr_err=sqrt(mean_squared_error(y_act, y_pred))

print('Mean absolute error: ', mean_abs_err)
print('Root Mean squared error: ', root_mean_sqr_err)

C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Mean absolute error:  0.25742239529883504
Root Mean squared error:  0.3796847578021573


In [63]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

y=train_data_final['Score']
X=train_data_final.drop(labels=['Score'], axis=1)

# Split the train data to train-test split
X_train, X_test, y_train, y_test=train_test_split(X, y, train_size=0.8, random_state=45)

# Training model object
regressor=GradientBoostingRegressor()
model_obj=regressor.fit(X_train, y_train)
y_pred=model_obj.predict(X_test)
y_act=y_test
mean_abs_err=mean_absolute_error(y_act, y_pred)
root_mean_sqr_err=sqrt(mean_squared_error(y_act, y_pred))

print('Mean absolute error: ', mean_abs_err)
print('Root Mean squared error: ', root_mean_sqr_err)

C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Mean absolute error:  0.2513746984276792
Root Mean squared error:  0.36701487306977854


In [21]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

y=train_data_final['Score']
X=train_data_final.drop(labels=['Score'], axis=1)

# Split the train data to train-test split
X_train, X_test, y_train, y_test=train_test_split(X, y, train_size=0.8, random_state=45)

# Training model object
regressor=XGBRegressor()
model_obj_xgb=regressor.fit(X_train, y_train)
y_pred=model_obj_xgb.predict(X_test)
y_act=y_test
mean_abs_err=mean_absolute_error(y_act, y_pred)
root_mean_sqr_err=sqrt(mean_squared_error(y_act, y_pred))

print('Mean absolute error: ', mean_abs_err)
print('Root Mean squared error: ', root_mean_sqr_err)

C:\Users\Arunachalam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Mean absolute error:  0.2512569377176331
Root Mean squared error:  0.36714119973700055


### Predictions on the test dataset

In [ ]:
y_pred_final=model_obj_xgb.predict(test_data_final.drop(labels='Score', axis=1))

In [ ]:
y_pred_final

In [ ]:

root_mean_sqr_err_final=sqrt(mean_squared_error(y_act_score, y_pred_final))
sigmoid_value=sigmoid(root_mean_sqr_err_final)
print('Score: ',(1-sigmoid_value))

In [25]:
sigmoid_value=sigmoid(0.36714119973700055)
print('Score: ',(1-sigmoid_value))

Score:  0.40923198869023103


In [ ]:
1-0.88

### Grid Search - GBM

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

param_test1 = {'n_estimators':[50, 100, 300], 'min_samples_leaf':[10, 50, 70], 'learning_rate':[0.01, 0.05, 0.1],'max_depth ':[2, 3, 5]}

gbm = GradientBoostingRegressor(random_state=10)

gsearch1 = GridSearchCV(estimator=gbm, param_grid = param_test1, scoring='explained_variance', cv=5)

In [ ]:
gsearch1.fit(X_train, y_train.values)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
X_train.values

In [ ]:
X_train.head()